# C# Encryption 101
The very basics you need to avoid doing encryption wrong

## Introduction

<table>
<tr>
<td><img src="https://www.lytzen.name/assets/franslytzenpicture.jpg"/></td>
<td>Moving software to the cloud since 2006.

CTO and CISO at NewOrbit; architecting and implementing large and small-scale software for our customers, all running on Microsoft Azure.

Helping other development organisations make the most of Azure through being an Azure Gold Partner and Reseller (“Direct CSP” in the parlance).

Obsessed with security, performance and scalability. Also, penny-pincher in the cloud.</td>
</tr>
</table>

----
We help organisations make the most of Azure. One of the things we do is Security Reviews on what people do in the cloud and sometimes that involves looking at code that encrypts data. The honest truth is that the encryption functionality in C# is not userfriendly and people make mistakes with it all the time. 

---

### I asked on Twitter if there was an appetite for this talk

<iframe id="twitter-widget-0" scrolling="no" frameborder="0" allowtransparency="true" allowfullscreen="true" class="" style="width: 550px; height: 361px; " title="Twitter Tweet" src="https://platform.twitter.com/embed/Tweet.html?dnt=false&amp;embedId=twitter-widget-0&amp;features=eyJ0ZndfdHdlZXRfZWRpdF9iYWNrZW5kIjp7ImJ1Y2tldCI6Im9mZiIsInZlcnNpb24iOm51bGx9LCJ0ZndfcmVmc3JjX3Nlc3Npb24iOnsiYnVja2V0Ijoib2ZmIiwidmVyc2lvbiI6bnVsbH0sInRmd190d2VldF9yZXN1bHRfbWlncmF0aW9uXzEzOTc5Ijp7ImJ1Y2tldCI6InR3ZWV0X3Jlc3VsdCIsInZlcnNpb24iOm51bGx9LCJ0Zndfc2Vuc2l0aXZlX21lZGlhX2ludGVyc3RpdGlhbF8xMzk2MyI6eyJidWNrZXQiOiJpbnRlcnN0aXRpYWwiLCJ2ZXJzaW9uIjpudWxsfSwidGZ3X2V4cGVyaW1lbnRzX2Nvb2tpZV9leHBpcmF0aW9uIjp7ImJ1Y2tldCI6MTIwOTYwMCwidmVyc2lvbiI6bnVsbH0sInRmd19kdXBsaWNhdGVfc2NyaWJlc190b19zZXR0aW5ncyI6eyJidWNrZXQiOiJvZmYiLCJ2ZXJzaW9uIjpudWxsfSwidGZ3X3VzZXJfZm9sbG93X2ludGVudF8xNDQwNiI6eyJidWNrZXQiOiJmb2xsb3ciLCJ2ZXJzaW9uIjpudWxsfSwidGZ3X3R3ZWV0X2VkaXRfZnJvbnRlbmQiOnsiYnVja2V0Ijoib2ZmIiwidmVyc2lvbiI6bnVsbH19&amp;frame=false&amp;hideCard=false&amp;hideThread=false&amp;id=1544268086076907521&amp;lang=en&amp;origin=https%3A%2F%2Fpublish.twitter.com%2F%3Fquery%3Dhttps%253A%252F%252Ftwitter.com%252Fflytzen%252Fstatus%252F1544268086076907521%26widget%3DTweet&amp;sessionId=59e9c3c5a849f572513093364300882fb644a37f&amp;theme=light&amp;widgetsVersion=b45a03c79d4c1%3A1654150928467&amp;width=550px" data-tweet-id="1544268086076907521"></iframe>

## 1: It's bytes all the way

Encryption functions are *maths* functions - they operate on numbers. They don't really understand things such as text, which is why all the inputs and outputs are *byte arrays* because they can all be treated as, and be broken into, *numbers* of differing sizes. This includes the encryption key, the payload and the encrypted content. 

Having a decent understanding of byte arrays, streams *and stream writers* as well as how to convert between text and bytes is essential to be able to use the encryption APIs well.

<img src="https://www.alpharithms.com/wp-content/uploads/338/bit-vs-byte-illustration.jpg" style="width:400px;">

### A little detour into text and bytes and streams

This is a big subject - I can only give you the very basics here.

#### Text has to be converted to and from bytes using a specific encoding standard

- Basically, a particular byte (or sequence of bytes) in a file or sent over the wire may refer to *different* letters, depending on the standard
- If in doubt, use UTF-8.
- Your string variable is, of course, internally stored as a sequence of bytes - but it's (effectively) UTF-16, which you probably don't want to use.

In [ ]:
var data = "Hello Ægir 个 😮";

var asciiBytes = Encoding.ASCII.GetBytes(data);
var utf8Bytes = Encoding.UTF8.GetBytes(data);
var utf16Bytes = Encoding.Unicode.GetBytes(data);

Console.WriteLine("ASCII: {0}", BitConverter.ToString(asciiBytes));
Console.WriteLine("UTF8:  {0}", BitConverter.ToString(utf8Bytes));
Console.WriteLine("UTF16: {0}", BitConverter.ToString(utf16Bytes));
Console.WriteLine("-----------------------");
Console.WriteLine("ASCII: {0}", Encoding.ASCII.GetString(asciiBytes));
Console.WriteLine("UTF8:  {0}", Encoding.UTF8.GetString(utf8Bytes));
Console.WriteLine("UTF16: {0}", Encoding.Unicode.GetString(utf16Bytes));

ASCII: 48-65-6C-6C-6F-20-3F-67-69-72-20-3F-20-3F-3F
UTF8:  48-65-6C-6C-6F-20-C3-86-67-69-72-20-E4-B8-AA-20-F0-9F-98-AE
UTF16: 48-00-65-00-6C-00-6C-00-6F-00-20-00-C6-00-67-00-69-00-72-00-20-00-2A-4E-20-00-3D-D8-2E-DE
-----------------------
ASCII: Hello ?gir ? ??
UTF8:  Hello Ægir 个 😮
UTF16: Hello Ægir 个 😮


#### Streams are kinda similar to byte arrays (at least for our purpose today)

- They both "hold" a sequence of bytes
- You can usually just keep writing to Stream, wheras you have to pre-declare the length of a byte array
- Streams *may* be "forward only"
- Streams are a very broad abstraction that covers too many things

### Back to the bytes

#### It looks a bit like this when you remove the nice syntactical sugar:

In [ ]:
// This code is not tested, and will probably not work and you almost certainly shouldn't use it as-is!!
using System;
using System.Security.Cryptography;

var key = new byte[32] { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31 };
var input = new byte[16] { 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15 };
var outputLength = (((input.Length / 16) + 1) * 16);
var output = new byte[outputLength];

using (var aes = Aes.Create())
{
    var encryptor = aes.CreateEncryptor();

    var inputLength = input.Length;
    var initialBlocks = (inputLength - 1) / 16;  
    for (int i = 0; i < initialBlocks; i++)
    {
        var position = i * 16;
        encryptor.TransformBlock(input, position, 16, output, position);
    }

    var finalBlockStart = initialBlocks * 16;
    var finalBlockLength = inputLength - (initialBlocks * 16);
    var finalBlock = encryptor.TransformFinalBlock(input, finalBlockStart, finalBlockLength);

    finalBlock.CopyTo(output, finalBlockStart);
}

#### That's rather ugly, so in practice you would tend to do something more like this. 

In [ ]:
// This example encrypts text held in memory! If your data is already in binary format - or just in a file - you can do it a little bit simpler
// Most of the layers here are about converting between binary and string formats! The crypto part is simpler. I'll come back to that.
using System.Security.Cryptography;
using System.IO;

var key = Encoding.ASCII.GetBytes("MySuperSecretKeyThatIs32BytesLon"); // Using text to create a key as bytes - not ideal!
var data = "Hello Ægir 个 😮";

byte[] encryptedBytes;

// Encrypt the data
using (var aes = Aes.Create())
using (var encryptor = aes.CreateEncryptor(key, aes.IV)) // Provide your own key, but use the generated IV
using (var encryptedStream = new MemoryStream())
using (var cryptoStream = new CryptoStream(encryptedStream, encryptor, CryptoStreamMode.Write))
using (var streamWriter = new StreamWriter(cryptoStream, Encoding.UTF8)) // This handles converting your data from text to bytes
{
    encryptedStream.Write(aes.IV, 0, aes.IV.Length); // Write the IV to the stream
    streamWriter.Write(data); // Encrypt the data
    streamWriter.Flush();
    cryptoStream.FlushFinalBlock();
    encryptedBytes = encryptedStream.ToArray();
}

## 2: Use Symmetric encryption for encrypting data

- Symmetric means you use the same key for encryption and decryption  
    -  It is very fast and can handle large volumes of data  
- Assymetric encryption uses different keys for encryption and decryption
    - It is slow and, in practice, can only encrypt very small bits of data
    - It has many other cryptographic uses


In the bonus content I will explain how you can combine them.

## 3: Use AES for symmetric Encryption

- AES is the symmetric encryption algorithm you should use by default. Cryptographic experts may choose others for specific purposes - but unless that's you, stick to AES.

- AES uses Rijndael - yes. But that doesn't mean you should directly use Rijndael. In fact, it has now been [marked as obsolete](https://github.com/dotnet/runtime/issues/46930). Use AES directly and let .Net handle the defaults.

**Don't do this:**

In [ ]:
using System.Security.Cryptography;

var rijndaelEncryptoer = Rijndael.Create();

**Do this:**

In [ ]:
using System.Security.Cryptography;

var encryptor = Aes.Create();

## 4: Leave the default AES settings alone

Unfortunately, the C# implementation of AES allows you to play with all the sharp knives. Do *not* change the defaults - Microsoft have already set them to their recommended values.

<img src="https://thumbs.dreamstime.com/b/little-baby-boy-reaching-hand-sharp-knife-kitchen-table-don-t-leave-kids-alone-little-baby-boy-reaching-hand-sharp-159366137.jpg" />

*Bad*:

In [ ]:
using System.Security.Cryptography;

using (var aes = Aes.Create())
{
    aes.BlockSize = 128;
    aes.Mode = CipherMode.ECB;
    aes.Padding = PaddingMode.ANSIX923;
    //.... do encryption work
}

*Good*:

In [ ]:
using System.Security.Cryptography;

using (var aes = Aes.Create())
{
    //.... do encryption work
}

## 5: The IV Value is important!

- The "Initialisation Vector" (IV) is like a random salt or a seed.
- Let AES create a new, random value for each item you encrypt
  - Do *not* set it to a fixed value!
- Keep the IV value with the item (it is not secret) so you can decrypt the content

In [ ]:
// This example encrypts text held in memory! If your data is already in binary format - or just in a file - you can do it a little bit simpler
// Most of the layers here are about converting between binary and string formats! The crypto part is simpler. I'll come back to that.
using System.Security.Cryptography;
using System.IO;

var key = Encoding.ASCII.GetBytes("MySuperSecretKeyThatIs32BytesLon"); // Using text to create a key as bytes - not ideal!
var data = "Hello Ægir 个 😮";

byte[] encryptedBytes;

using (var aes = Aes.Create())
using (var encryptor = aes.CreateEncryptor(key, aes.IV)) // Provide your own key, but use the generated IV
using (var encryptedStream = new MemoryStream())
using (var cryptoStream = new CryptoStream(encryptedStream, encryptor, CryptoStreamMode.Write))
using (var streamWriter = new StreamWriter(encryptedStream, Encoding.UTF8)) // This handles converting your data from text to bytes
{
    cryptoStream.Write(aes.IV, 0, aes.IV.Length); // Write the IV to the stream
    streamWriter.Write(data); // Encrypt the data
    streamWriter.Flush();
    cryptoStream.FlushFinalBlock();
    encryptedBytes = encryptedStream.ToArray();
}

## Examples

### Utility and intro

These examples all use the `ICryptoStream` interface. For some scenarios, it can be cleaner to use the `byte[]` interface - but it has other complexities to get your head around so I strongly recommend using the Stream based interface for your own sanity.

**Create a file we can encrypt**

In [ ]:
using System.IO;
// Utility function
public static void DeleteIfExists(this string fileName)
{
    if (File.Exists(fileName))
    {
        File.Delete(fileName);
    }
}


In [ ]:
using System.IO;
// Write some text to a file so we can look at stuff

var inputFileName = "input.txt";

inputFileName.DeleteIfExists();

using (var inputFile = File.CreateText(inputFileName)) // CreateText always uses UTF8!
{
    inputFile.WriteLine("Hello Ægir 个 😮");
}

**Create an encryption key**

In [ ]:
using System.Security.Cryptography;
// This creates a random key and then Base64 encodes it so you can store it in config (be careful to do this safely!)
var keyAsBase64 = Convert.ToBase64String(Aes.Create().Key);

### Binary/Stream Encryption

In [ ]:
// This example uses files to avoid dealing with text-to-byte conversions and streamreaders/writers
// It works equally well with, say, a request stream in a web application as an input and, say, an Azure Blob stream for the output
// No need to use MemoryStream in the middle, nor to convert to/from text.
// If you do want to encrypt/decrypt text in memory, you need to do those conversions and use streamwriters/readers and a memorystream.
using System.Security.Cryptography;
using System.IO;

var encryptedFileName = "encrypted.bin";
var decryptedFileName = "decrypted.txt";
encryptedFileName.DeleteIfExists();
decryptedFileName.DeleteIfExists();

var key = Convert.FromBase64String(keyAsBase64);

// Encrypt the file
using (var inputStream = File.OpenRead(inputFileName)) // Opens the file as BINARY!
using (var outputStream = File.Create(encryptedFileName)) // Creates the output file as BINARY!
using (var aes = Aes.Create())
using (var encryptor = aes.CreateEncryptor(key, aes.IV)) // Provide your own key, but use the generated IV
using (var cryptoStream = new CryptoStream(outputStream, encryptor, CryptoStreamMode.Write))
{
    cryptoStream.Write(aes.IV, 0, aes.IV.Length); // Write the IV to the stream
    inputStream.CopyTo(cryptoStream); // Encrypt the data
    cryptoStream.FlushFinalBlock();
    outputStream.Flush();
}


// Decrypt the file
using (var inputStream = File.OpenRead(encryptedFileName)) // Opens the file as BINARY!
using (var outputStream = File.Create(decryptedFileName)) // Creates the output file as BINARY!
using (var aes = Aes.Create()) 
{
    byte[] iv = new byte[16];
    inputStream.Read(iv, 0, iv.Length); // Read the IV from the stream
    using (var decryptor = aes.CreateDecryptor(key, iv))
    using (var cryptoStream = new CryptoStream(inputStream, decryptor, CryptoStreamMode.Read))  
    {
        {
            cryptoStream.CopyTo(outputStream); // Note that the stream pointer is at byte 16 at this point
            outputStream.Flush();
        }
    }
}

// Check
var inputText = File.ReadAllText(inputFileName);
var decryptedText = File.ReadAllText(decryptedFileName);
Console.WriteLine("Input: {0}", inputText);
Console.WriteLine("Decrypted: {0}", decryptedText);


Input: Hello Ægir 个 😮

Decrypted: Hello Ægir 个 😮



### Text encryption

In [ ]:
using System.Security.Cryptography;
using System.IO;

var key = Convert.FromBase64String(keyAsBase64);
var data = "Hello Ægir 个 😮";

byte[] encryptedBytes;

// Encrypt the data
using (var aes = Aes.Create())
using (var encryptor = aes.CreateEncryptor(key, aes.IV)) // Provide your own key, but use the generated IV
using (var encryptedStream = new MemoryStream())
using (var cryptoStream = new CryptoStream(encryptedStream, encryptor, CryptoStreamMode.Write))
using (var streamWriter = new StreamWriter(cryptoStream, Encoding.UTF8)) // This handles converting your data from text to bytes
{
    encryptedStream.Write(aes.IV, 0, aes.IV.Length); // Write the IV to the stream
    streamWriter.Write(data); // Encrypt the data
    streamWriter.Flush();
    cryptoStream.FlushFinalBlock();
    encryptedBytes = encryptedStream.ToArray();
}

string decryptedText;

// Decrypt the data
// Note: when decrypting an in-memory byte[], there is a shortcut that bypassess the need for a stream, but I am sticking with streams for consistency
using (var aes = Aes.Create())
using (var inputStream = new MemoryStream())
using (var outputStream = new MemoryStream())
{
    inputStream.Write(encryptedBytes, 0, encryptedBytes.Length); // Write the encrypted data to the stream
    inputStream.Position = 0;
    var iv = new byte[16];
    inputStream.Read(iv, 0, iv.Length); // Read the IV from the stream

    using (var decryptor = aes.CreateDecryptor(key, iv))
    using (var cryptoStream = new CryptoStream(inputStream, decryptor, CryptoStreamMode.Read))
    {
        cryptoStream.CopyTo(outputStream); // Decrypt the data - note that the pointer is at byte 16 at this point
        outputStream.Flush();
    }
    outputStream.Position = 0; // Reset the pointer to the start of the stream - not all streams can do this!!
    using (var streamReader = new StreamReader(outputStream, UTF8Encoding.UTF8)) // This handles converting your data from bytes to text
    {
        decryptedText = streamReader.ReadToEnd();
    }
}

Console.WriteLine("Input: {0}", data);
Console.WriteLine("Decrypted: {0}", decryptedText);

Input: Hello Ægir 个 😮
Decrypted: Hello Ægir 个 😮


# Bonus content

## A. Use the envelope method

This is a well-known pattern implemented, for example, by Azure Blob Storage Client-Side Encryption.

#### The problem

- Using the *same* symmetric encryption for every item increases risk:
  - If someone gets that key, it is game over.
  - Despite the IV, there *may* be ways to deduce the key if you have many items encrypted with the same symmetric key.

#### The solution

- Using a unique Key for every item you encrypt (Aes creates one for you automatically)
- Encrypt the Symmetric key with an Asymetric key (i.e. public/private key pair)
- Store the *encrypted* symmetric key alongside the IV and the encrypted content.
- Ideally use Azure Keyvault (or similar) to encrypt the symmetric key.
  - Also enable Key Rotation on the key used so you change the asymetric key regularly
  - Store a reference to the correct asymetric key so you can decrypt the symmetric key when you want to read the content.

## B. Authenticated Encryption

### The problem

In [ ]:
using System.Security.Cryptography;
using System.IO;

var key = Aes.Create().Key;
var data = "Hello Ægir 个 😮";

byte[] encryptedBytes;

// Encrypt the data
using (var aes = Aes.Create())
using (var encryptor = aes.CreateEncryptor(key, aes.IV)) // Provide your own key, but use the generated IV
using (var encryptedStream = new MemoryStream())
using (var cryptoStream = new CryptoStream(encryptedStream, encryptor, CryptoStreamMode.Write))
using (var streamWriter = new StreamWriter(cryptoStream, Encoding.UTF8)) // This handles converting your data from text to bytes
{
    encryptedStream.Write(aes.IV, 0, aes.IV.Length); // Write the IV to the stream
    streamWriter.Write(data); // Encrypt the data
    streamWriter.Flush();
    cryptoStream.FlushFinalBlock();
    encryptedBytes = encryptedStream.ToArray();
}

// LET'S MESS WITH THE ENCRYPTED DATA!!
encryptedBytes[18] = 0;
encryptedBytes[19] = 0;
encryptedBytes[20] = 0;

string decryptedText;

// Decrypt the data
// Note: when decrypting an in-memory byte[], there is a shortcut that bypassess the need for a stream, but I am sticking with streams for consistency
using (var aes = Aes.Create())
using (var inputStream = new MemoryStream())
using (var outputStream = new MemoryStream())
{
    inputStream.Write(encryptedBytes, 0, encryptedBytes.Length); // Write the encrypted data to the stream
    inputStream.Position = 0;
    var iv = new byte[16];
    inputStream.Read(iv, 0, iv.Length); // Read the IV from the stream

    using (var decryptor = aes.CreateDecryptor(key, iv))
    using (var cryptoStream = new CryptoStream(inputStream, decryptor, CryptoStreamMode.Read))
    {
        cryptoStream.CopyTo(outputStream); // Decrypt the data - note that the pointer is at byte 16 at this point
        outputStream.Flush();
    }
    outputStream.Position = 0; // Reset the pointer to the start of the stream - not all streams can do this!!
    using (var streamReader = new StreamReader(outputStream, UTF8Encoding.UTF8)) // This handles converting your data from bytes to text
    {
        decryptedText = streamReader.ReadToEnd();
    }
}

Console.WriteLine("Input: {0}", data);
Console.WriteLine("Decrypted: {0}", decryptedText);

Input: Hello Ægir 个 😮
Decrypted: q��م�����j@#zZ��Z䛘�


Note that the code *didn't blow up*. According to the experts, this leads to a number of practically-exploitable vulnerabilities (see [Security Driven .Net](https://securitydriven.net/) and the advice is therefore to use "authenticated encryption", which will detect any tampering.

### The solution

- Authenticated Encryption in essence means you create a Hash-based Message Authentication Code (HMAC) for the encrypted content (incl. the IV) using, ideally, a separate secret.

- The book referenced above explains how to do this.

- These days I would personally recommend using Azure Keyvault or similar to do this HMAC signing instead.

- One major challenge is that you need to keep all of the encrypted content in memory so you can create the HMAC. 

## Shameless plug #1

I am working on an open-source [library](https://github.com/NewOrbit/NewOrbit.Azure.KeyVault.Cryptography) that encapsulates all of the above and takes all of the choices away. It uses Azure Keyvault to implement both the envelope method and authenticated encryption (and you don't get a choice if you want it).

It's work in progress, but this talk has prompted me to get it finished and released real-soon-now.

## Shameless plug #2

<img src="https://neworbit.co.uk/uploads/NewOrbit-logo-strapline_white.svg" style="width:400px;"/>

My company, NewOrbit is an Azure partner who specialises in helping developers make the most of Azure. Software houses buy their Azure through us for the same price they get it from Microsoft. Our customers get access to our Azure expertise at reduced rates, in particular around cost optimisation and security.

<img src="https://neworbit.co.uk/uploads/NewOrbit-MS-Partner-Competencies.gif"  />

## References

- [Security Driven .Net](https://securitydriven.net/) book. An oldie but still awesome.
- [Applied Cryptography in .NET and Azure Key Vault](https://www.apress.com/us/applied-cryptography-in--net-and-azure-key-vault/16427394) book
- [Characters, Symbols and the Unicode Miracle - Computerphile](https://www.youtube.com/watch?v=MijmeoH9LT4) - video explaining UTF text encoding
- [C# In depth - Unicode](https://www.csharpindepth.com/articles/Unicode) - excellent description of the whole text encoding thing